Planeo traducir los datasets en ingles a otros idiomas a traves de DeepL. Para ello, me hice una cuenta premium en la cual DeepL me permite traducir documentos de hasta 1M de caracteres (en realidad un poco menos, pero esto no lo anuncian).

Como algunos de los datasets tienen mas de 1M de caracteres, el plan es partirlos en varios documentos de hasta cierto umbral (850k), mandarlos a traducir a DeepL, y luego unificarlos con los sentimientos de los datasets originales en el mismo orden.

Ahora prosigo a desarmarlos.

In [17]:
file1 = 'datasetCrowdflower2M' #3 millones de caracteres. debe ser partido en 3
file2 = 'datasetElectoral2M' #400k no hay necesidad de partirlo para Deepl
file3 = 'datasetEmoInt2M' #400k no hay necesidad de partirlo
file4 = 'datasetTEC2M'#1 millon 40k. Debe ser partido en 2
fileUnificado = 'datasetUnificado'

chosen_file = file4
dic_partitions = {file1:3, file2:1, file3:1, file4:2}
partitions = dic_partitions[chosen_file]

In [18]:
import pandas as pd

df = pd.read_csv(chosen_file+'.csv', sep='\t', encoding='utf-16')

In [19]:
df.head()

,text,emotion
0,Thinks that @melbahughes had a great 50th birt...,7
1,Como una expresión tan simple una sola oración...,3
2,the moment when you get another follower and y...,1
3,eww.. my moms starting to make her annual rum ...,4
4,If ur heart hurts all the time for tht person ...,1


In [15]:
from docx import Document

path='toTranslate/'

list_documents = list()
for i in range(partitions):
    list_documents.append(Document())

act_chars = 0
umbral_chars = 850000
index_doc = 0

for i in range(len(df['text'])):
    list_documents[index_doc].add_paragraph(df['text'][i])
    act_chars+=len(df['text'][i])
    
    #Si la siguiente iteracion supera el umbral de deepl, guardo el archivo compeltoado y sigo imprimiendo en el proximo archivo
    if act_chars + len(df['text'][i]) > umbral_chars or i==len(df['text'])-1:
        list_documents[index_doc].save(path+chosen_file+'- a traducir - parte'+str(index_doc+1)+'.docx')
        index_doc+=1
        act_chars = 0

IndexError: list index out of range

Teniendo los datasets 1 y 4 desarmados pero traducidos, prosigo a rearmarlos.

In [1]:
import numpy as np
import pandas as pd
import docx

path_united_en = 'datasets/united/en/' #Dataset ya armado con etiquetas en ingles

In [3]:
#Armo el dataset Crowdflower
df_en1 = pd.read_csv(path_united_en+'datasetCrowdflower2-en.csv', encoding='utf-16', sep='\t')


for lang in ['es', 'pt']:
    list_texts = list()
    list_emotions = list()
    idx_emotion = 0
    print(lang)
    path_parts = 'datasets/parts/'+lang+'/' #Dataset con las traducciones separadas en varios archivos sin etiquetas
    path_united_translated = 'datasets/united/'+lang+'/' #Dataset donde iran las traducciones unidas con etiquetas
    for i in range(1,5):
        d1 = docx.Document(path_parts + 'datasetCrowdflower2M- a traducir - parte' + str(i) + ' '+lang+'.docx')
        for j in range(len(d1.paragraphs)):
            text = d1.paragraphs[j].text.replace('\t', ' ') #Elimino la UNICA tabulacion que aparece en todo el dataset
                                                            # leccion del dia: SIEMPRE hay que eliminar
                                                            # el caracter que usas como separador
              #Reemplazo los textos que considero que tienen errores
            list_texts.append(d1.paragraphs[j].text)    #Por un string que interpretare como no valido
            list_emotions.append(df_en1['emotion'][idx_emotion])
            idx_emotion+=1
        print('Finalizada la parte '+ str(i))
    print('Creando diccionario')
    dic_es = {'text':list_texts, 'emotion':list_emotions}
    print('Creando DataFrame')
    complete_df1 = pd.DataFrame(dic_es)
    complete_df1.count()
    complete_df1.to_csv(path_united_translated + 'datasetCrowdflower2-'+lang+'.csv', sep='\t',
                        encoding='utf-16', index=False)

es
Finalizada la parte 1
Finalizada la parte 2
Finalizada la parte 3
Finalizada la parte 4
Creando diccionario
Creando DataFrame
pt
Finalizada la parte 1
Finalizada la parte 2
Finalizada la parte 3
Finalizada la parte 4
Creando diccionario
Creando DataFrame


In [7]:
#Rearmo el dataset Electoral

list_texts = list()
list_emotions = list()

df_en2 = pd.read_csv(path_united_en+'datasetElectoral2M.csv', encoding='utf-16', sep='\t')

d2 = docx.Document(path_parts + 'datasetElectoral2M- a traducir ' + lang +'.docx')

for j in range(len(d2.paragraphs)):
    list_texts.append(d2.paragraphs[j].text)
    list_emotions.append(df_en2['emotion'][j])

dic_es = {'text':list_texts, 'emotion':list_emotions}
complete_df2 = pd.DataFrame(dic_es)
complete_df2.to_csv(path_united_translated + 'datasetElectoral'+lang+'.csv', sep='\t', encoding='utf-16', index=False)

In [22]:
#Rearmo el dataset Emoint

list_texts = list()
list_emotions = list()

df_en3 = pd.read_csv(path_united_en+'datasetEmoInt2-en.csv', encoding='utf-16', sep='\t')

d3 = docx.Document(path_parts + 'datasetEmoInt2M- a traducir '+lang+'.docx')

for j in range(len(d3.paragraphs)):
    list_texts.append(d3.paragraphs[j].text)
    list_emotions.append(df_en3['emotion'][j])
dic_es = {'text':list_texts, 'emotion':list_emotions}
complete_df3 = pd.DataFrame(dic_es)
complete_df3.to_csv(path_united_translated + 'datasetEmoInt'+lang+'.csv', sep='\t', encoding='utf-16', index=False)

In [17]:
#Rearmo el dataset Tec

df_en4 = pd.read_csv(path_united_en+'datasetTec2-en.csv', encoding='utf-16', sep='\t')

for lang in ['es']:
    list_texts = list()
    list_emotions = list()
    idx_emotion = 0
    print(lang)
    path_parts = 'datasets/parts/'+lang+'/' #Dataset con las traducciones separadas en varios archivos sin etiquetas
    path_united_translated = 'datasets/united/'+lang+'/' #Dataset donde iran las traducciones unidas con etiquetas
    for i in range(1,3):
        d4 = docx.Document(path_parts + 'datasetTEC2M- a traducir - parte' + str(i) + ' '+lang+'.docx')
        for j in range(len(d4.paragraphs)):
            text = d4.paragraphs[j].text.replace('\t', ' ') #Elimino la UNICA tabulacion que aparece en todo el dataset
                                                            # leccion del dia: SIEMPRE hay que eliminar
                                                            # el caracter que usas como separador
              #Reemplazo los textos que considero que tienen errores
            list_texts.append(d4.paragraphs[j].text)    #Por un string que interpretare como no valido
            list_emotions.append(df_en4['emotion'][idx_emotion])
            idx_emotion+=1
        print('Finalizada la parte '+ str(i))
    print('Creando diccionario')
    dic_es = {'text':list_texts, 'emotion':list_emotions}
    print('Creando DataFrame')
    complete_df4 = pd.DataFrame(dic_es)
    complete_df4.to_csv(path_united_translated + 'datasetTec'+lang+'.csv', sep='\t', encoding='utf-16', index=False)

es
Finalizada la parte 1
Finalizada la parte 2
Creando diccionario
Creando DataFrame


Los datasets son muy pequeños. El dataset unificado deberia ser de 64147 entradas

In [25]:
lang = 'pt'

pathTrans = 'datasets/united/'+lang+'/'
fileTrans1 = pathTrans + 'datasetCrowdflower2-'+lang+'.csv'
fileTrans2 = pathTrans + 'datasetElectoral2-'+lang+'.csv'
fileTrans3 = pathTrans + 'datasetEmoInt2-'+lang+'.csv'
fileTrans4 = pathTrans + 'datasetTec2-'+lang+'.csv'

In [27]:
import pandas as pd

df1 = pd.read_csv(fileTrans1, encoding='utf-16', sep='\t')
df2 = pd.read_csv(fileTrans2, encoding='utf-16', sep='\t')
df3 = pd.read_csv(fileTrans3, encoding='utf-16', sep='\t')
df4 = pd.read_csv(fileTrans4, encoding='utf-16', sep='\t')

In [6]:
print(df1.count())
print(df2.count())
print(df3.count())
print(df4.count())

text       39727
emotion    39739
dtype: int64
text       4055
emotion    4055
dtype: int64
text       7097
emotion    7101
dtype: int64
text       13249
emotion    13252
dtype: int64


In [28]:
dfUnificado = df1.append(df2).append(df3).append(df4)

In [29]:
print(dfUnificado.count())
print(len(dfUnificado))

text       64133
emotion    64147
dtype: int64
64147


In [30]:
len(df1) #Deberia ser de 39739

39739

In [31]:
len(df2) #Deberia ser de 4055

4055

In [32]:
len(df3) #Deberia ser de 7101

7101

In [33]:
len(df4) #Este deberia ser de 13252

13252

In [34]:
dfUnificado.to_csv(pathTrans + 'datasetUnificado2'+lang+'.csv', sep='\t', encoding='utf-16', index=False)

In [35]:
print(len(df1))
print(len(df2))
print(len(df3))
print(len(df4))

39739
4055
7101
13252


In [23]:
pathEn = 'datasets/united/en/'
fileEn = pathEn + 'datasetUnificado2-en.csv'

dfEn = pd.read_csv(fileEn, encoding='utf-16', sep='\t')

dfEn.tail()

,text,emotion
64142,About to have a movie night with my booboo @je...,3
64143,@TheBodyShopUK Knowing my dissertation will be...,1
64144,hospital tomorrow morning strapped with wires ...,1
64145,Work is soooo slow ready to have a great saturday,1
64146,You realize that by choosing joy every single ...,1


In [36]:
dfUnificado.tail()

,text,emotion
13247,Prestes a ter uma noite de cinema com meu boob...,3
13248,@TheBodyShopUK Sabendo que minha dissertação s...,1
13249,hospital amanhã de manhã amarrado com arames e...,1
13250,"O trabalho é muito lento, pronto para ter um ó...",1
13251,Você percebe que ao escolher a alegria em cada...,1
